In [0]:
from pyspark.sql import SparkSession # Puerta de entrada para trabajar con spark <-- SIEMPRE DEBEMOS IMPORTAR LA LLAVE MAESTRA QUE INICIA TODO.
from pyspark.sql.functions import *  # Funciones propias del módulo SQL de Spark, para trabajar sobre Dataframes.
from pyspark.sql.types import *
spark = SparkSession.builder.appName("ApacheSparkDatabricksApuntes").getOrCreate() 
"""
^          ^__________^        ^_________^                               ^
|                |                   |                                   | 
Variable   Constructor de Sesión   Nombre Aplicación       Evita conflicto del SparkSession"""


### Explicación de Catálagos - Esquemas - Volumenes/Tablas

#### JERARQUÍA UNITY CATALOG (CREACIÓN)

In [0]:
# Crear un CATÁLAGO (carpeta o conjunto de carpetas que almacenará los archivos):
# >> Usando SQL EDITOR (SINTAXIS): 
    ## CREATE CATALOG IF NOT EXISTS nombre_catalago; 
# >> Usando PySpark (SINTAXIS): 
    ## spark.sql("CREATE CATALOG IF NOT EXISTS nombre_catalago")

#### ➡️ Crearemos un catálago para los ejemplos que explicaré mas adelante
spark.sql("CREATE CATALOG IF NOT EXISTS explicacion_unity_catalog")
print("Se creo el catálago para los ejemplos!!!")

In [0]:
# Crear un ESQUEMA (agrupación de archivos basado en el objetivo del proyecto):
# >> Usando SQL EDITOR (SINTAXIS): 
    ## CREATE SCHEMA IF NOT EXISTS nombre_catalago.NombreEsquema;
# >> Usando PySpark (SINTAXIS): 
    ## spark.sql("CREATE SCHEMA IF NOT EXISTS nombre_catalago.NombreEsquema;")

#### ➡️ Crearemos un esquema para los ejemplos que explicaré mas adelante
spark.sql("CREATE SCHEMA IF NOT EXISTS explicacion_unity_catalog.ejemplos")
print("Se creo el esquema para los ejemplos!!!")

In [0]:
# Crear un VOLUMEN (contenedores de archivos físicos sin esquema tabular):
# >> Usando SQL EDITOR (SINTAXIS): 
    ## CREATE VOLUME IF NOT EXISTS nombre_catalago.nombre_esquema.NombreVolumen;
# >> Usando PySpark (SINTAXIS): 
    ## spark.sql("CREATE VOLUME IF NOT EXISTS nombre_catalago.nombre_esquema.NombreVolumen;")

#### ALMACENAR/LEER OBJETOS EN UNITY CATALOG

In [0]:
# PATH PARA ALMACENAR OBJETOS EN VOLUMENES (RECOMENDADO POR GOBERNANZA DE UNITY CATALOG): 
# SINTAXIS:
"/Volumes/nombre_catalago/nombre_esquema/nombre_volumen" 
# EJEMPLO:
"/Volumes/workspace/exercises/volumen_dataframe"

# USAREMOS DE EJEMPLO UN DATAFRAME (TITANIC):
dataframe_titanic = spark.sql("SELECT * FROM workspace.exercises.titanic") ##⬅️ Explicaremos más adelante

##### ARCHIVOS PARQUET

Parquet organiza los datos por columnas, lo que permite una mayor compresión y consultas más
rápidas, especialmente cuando solo se requieren algunas columnas específicas.

###### ALMACENAR

In [0]:
# Almacenar en un archivo parquet el dataframe_titanic (sobrevivientes==1)
df_sobrevivientes = dataframe_titanic.filter(col("survived")==1)
# df_sobrevivientes.show()

#### ➡️ Crearemos un volumen para almacenar el archivo .parquet del dataframe_titanic
spark.sql("CREATE VOLUME IF NOT EXISTS explicacion_unity_catalog.ejemplos.parquet_titanic")
print("Se creo el volumen para el archivo parquet del dataframe_titanic!!!")

#### 📌 Almacenamos en un .parquet el dataframe filtrado
df_sobrevivientes.write.mode("overwrite").parquet("/Volumes/explicacion_unity_catalog/ejemplos/parquet_titanic")
print("Archivo parquet creado y almacenado en el volúmen!!!")


###### LEER

In [0]:
# Leer el archivo parquet del dataframe_titanic filtrado creado anteriormente
parquet_titanic = spark.read.parquet("/Volumes/explicacion_unity_catalog/ejemplos/parquet_titanic")
parquet_titanic.show(3)
print("Lectura de archivo parquet correctamente")


##### ARCHIVOS CSV

Para este ejemplo leeremos una tabla desde Unity Catalog, lo almacenamos como CSV en el volumen
y luego leemos ese CSV.

###### ALMACENAR

In [0]:
# Almacenar en un archivo csv el dataframe_titanic (sobrevivientes==1)
df_sobrevivientes_table = dataframe_titanic.filter(col("survived")==1)
# df_sobrevivientes_table.show()

#### ➡️ Convertir la tabla proveniente de Unity Catalog a un CSV (Solo para ejemplos didácticos)

#### Creamos el volúmen que almacenará el archivo CSV 
spark.sql("CREATE VOLUME IF NOT EXISTS explicacion_unity_catalog.ejemplos.csv_titanic")
print("Se creó correctamente el volumen!!!")

#### 📌 Almacenar en un CSV 
df_sobrevivientes_table.write.mode("overwrite").option("header","true").option("inferSchema","true").csv("/Volumes/explicacion_unity_catalog/ejemplos/csv_titanic")
print("Se almacenó correctamente el CSV en el volúmen")

###### LEER

In [0]:
# Leer el archivo CSV del dataframe_titanic filtrado creado anteriormente.
csv_titanic = spark.read.option("header","true").option("inferSchema","true").csv("/Volumes/explicacion_unity_catalog/ejemplos/csv_titanic")
csv_titanic.show(3)
print("Se leyó correctamente el archivo CSV!!!")

##### ARCHIVOS DELTA / DELTA TABLES (MAYOR OPTIMIZACIÓN)

Permiten una mejor optimización en la lectura de la información, debido que trabajan
sobre delta lakes (motores de procesamiento muy eficientes basados en archivos parquet). Sin embargo, existe una diferencia en como
se almacenan y leen.

###### ALMACENAR

In [0]:
#### ➡️ ARCHIVOS DELTA (NO SON ENTIDADES GOBERNADAS POR UNITY CATALOG, POR ENDE NO PUEDEN INTERACTUAR MEDIANTE SPARK SQL)
## Es el modo por defecto que Databricks establece a cada archivo subido desde Catalog.

###📌 Almacenamos en archivo delta
df_sobrevivientes = dataframe_titanic.filter(col("survived")==1)
# df_sobrevivientes.show(3)

## ➡️ Creamos el volumen
spark.sql("CREATE VOLUME IF NOT EXISTS explicacion_unity_catalog.ejemplos.archivo_delta_titanic")
print("Volumen creado correctamente!!!")
df_sobrevivientes.write.format("delta").mode("overwrite").save("/Volumes/explicacion_unity_catalog/ejemplos/archivo_delta_titanic")
print("Archivo delta creado correctamente!!!")


###📌 Almacenamos en delta table
df_sobrevivientes.write.format("delta").mode("overwrite").saveAsTable("explicacion_unity_catalog.ejemplos.delta_titanic")
print("Delta table creado correctamente!!!")


###### LEER

In [0]:
### ➡️ Leer el archivo delta y la delta table del dataframe_titanic filtrado creado anteriormente.

#### 📌 LECTURA DE ARCHIVO DELTA
archivo_delta_titanic = spark.read.format("delta").load("/Volumes/explicacion_unity_catalog/ejemplos/archivo_delta_titanic/")
archivo_delta_titanic.show(3)

#### 📌 LECTURA DE DELTA TABLE (RECOMENDADA)
delta_table_titanic = spark.sql("SELECT * FROM explicacion_unity_catalog.ejemplos.delta_titanic")
delta_table_titanic.show(3)
print("Lectura correcta de ambas formas delta!!")